In [ ]:
## Init
import urllib.request
import io

import numpy as np
from PIL import Image

from keras.preprocessing.image import load_img, img_to_array, array_to_img
from keras import backend as K
from keras.applications.vgg16 import preprocess_input, VGG16
from IPython.display import display
from keras.optimizers import SGD, Adam

def postprocess_array(x):
    t = x.copy()
    t = t[0, :, :, :]
    vgg_mean = [103.939, 116.779, 123.68]
    for i in range(3):
        t[:, :, i] += vgg_mean[i]
    t = t[:,:,::-1]
    t = np.clip(t, 0, 255).astype('uint8')
    return t

def load_image_online(filename):
    with urllib.request.urlopen(filename) as url:
        f = io.BytesIO(url.read())
    image = Image.open(f).convert("RGB")
    return image

def preprocess_image(image_path, image_size = None):
    image = load_image_online(image_path)
    if image_size is not None:
        if isinstance(image_size, int):
            w, h = image.size
            image = image.resize((image_size, image_size * h // w))
        elif isinstance(image_size, tuple):
            image = image.resize(image_size)
    image = np.array(image)
    image = np.array([image])
    image = preprocess_input(image)
    return image

In [ ]:
PLACEHOLDER_CONTENT_PATH = "http://apex.sjtu.edu.cn/public/files/avatar/20180912/lyk.jpg"
PLACEHOLDER_STYLE_PATH = "https://uploads7.wikiart.org/images/giorgio-de-chirico/hector-and-andromache-1912.jpg!Large.jpg"
PLACEHOLDER_IMAGE_SIZE = 256
PLACEHOLDER_CONTENT_WEIGHT = 1
PLACEHOLDER_STYLE_WEIGHT = 1e4
PLACEHOLDER_LEARNING_RATE = 10
PLACEHOLDER_ITERATIONS = 10

In [ ]:
## Run

content_image = preprocess_image(PLACEHOLDER_CONTENT_PATH, PLACEHOLDER_IMAGE_SIZE)
style_image = preprocess_image(PLACEHOLDER_STYLE_PATH, (content_image.shape[2], content_image.shape[1]))

content_input = K.constant(content_image)
style_input = K.constant(style_image)
output_image = K.variable(content_image)

input_tensor = K.concatenate([content_input, style_input, output_image], axis=0)
model = VGG16(input_tensor=input_tensor,
              weights='../models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
              , include_top=False)

output_dict = dict([(layer.name, layer.output) for layer in model.layers])

def get_content_loss(content_features, output_features):
    return 0.5 * K.sum(K.square(output_features - content_features))

layer_feat = output_dict['block4_conv2']

content_feat = layer_feat[0, :, :, :]

output_feat = layer_feat[2, :, :, :]

loss = PLACEHOLDER_CONTENT_WEIGHT * get_content_loss(content_feat, output_feat)

def get_gram_matrix(x):
    feature_matrix = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram_matrix = K.dot(feature_matrix, K.transpose(feature_matrix))
    return gram_matrix

def get_style_loss(style_features, output_features):
    G = get_gram_matrix(style_features)
    A = get_gram_matrix(output_features)
    
    channel_number = int(style_features.shape[2])
    size = int(style_features.shape[0]) * int(style_features.shape[1])
    return K.sum(K.square(G - A)) / (4.0 * (channel_number ** 2) * (size ** 2))

layer_names = ['block1_conv1', 'block2_conv1',
               'block3_conv1', 'block4_conv1',
               'block5_conv1']

for layer_name in layer_names:
    layer_feat = output_dict[layer_name]
    style_feat = layer_feat[1, :, :, :]
    output_feat = layer_feat[2, :, :, :]
    single_style_loss = get_style_loss(style_feat, output_feat)
    loss += (PLACEHOLDER_STYLE_WEIGHT / len(layer_names)) * single_style_loss
    
updater = Adam(lr = PLACEHOLDER_LEARNING_RATE).get_updates(loss, [output_image])

fit = K.function([], [loss, output_image], updater)

out = None
for i in range(PLACEHOLDER_ITERATIONS):
    f_val, out = fit([])
    img = postprocess_array(out)
    display(array_to_img(img))